# Multi-tenant Chat with Papers - Query papers
## Get keys and urls

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")



Weaviate Key:root-user-key
OpenAI API Key: sk-dummy-key-for-local-testing
OpenAI URL: http://host.docker.internal:11434


## Connect to Weaviate

In [10]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


## Vector search on tenants

In [3]:
papers = client.collections.get("Papers")

ten = papers.with_tenant("2212-10496")

response = ten.query.near_text(
    query="Unsupervised learning",
    limit=5,
)

for item in response.objects:
    print(item.properties["chunk"])

2020. Unsupervised cross-lingual representation learning at scale. In Proceedings of the 58th Annual Meeting of the Asso- ciation for Computational Linguistics , pages 8440– 8451, Online. Association for Computational Lin- guistics. Nick Craswell, Bhaskar Mitra, Emine Yilmaz, Daniel Campos, and Ellen M. V oorhees. 2020a. Overview of the trec 2019 deep learning track. Nick Craswell, Bhaskar Mitra, Emine Yilmaz, Daniel Fernando Campos, and Ellen M. V oorhees. 2020b. Overview of the trec 2020 deep learning track. ArXiv , abs/2003.07820. Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. 2019. BERT: Pre-training of deep bidirectional transformers for language under- standing. In Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers)
standard supervised sequence-to-sequence learning or more effectively with reinforcement learning (Ouyang et al., 2022). Con

## Generative Search with tenants

In [ ]:
papers = client.collections.get("Papers")

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=3,
    single_prompt="What does the following text describe: {chunk}",
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generative.text, '\n')

2020. Unsupervised cross-lingual representation learning at scale. In Proceedings of the 58th Annual Meeting of the Asso- ciation for Computational Linguistics , pages 8440– 8451, Online. Association for Computational Lin- guistics. Nick Craswell, Bhaskar Mitra, Emine Yilmaz, Daniel Campos, and Ellen M. V oorhees. 2020a. Overview of the trec 2019 deep learning track. Nick Craswell, Bhaskar Mitra, Emine Yilmaz, Daniel Fernando Campos, and Ellen M. V oorhees. 2020b. Overview of the trec 2020 deep learning track. ArXiv , abs/2003.07820. Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. 2019. BERT: Pre-training of deep bidirectional transformers for language under- standing. In Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers)
This text describes a research paper titled "BERT" that was presented at the 2019 Conference of the North American Chapter o

In [12]:
papers = client.collections.get("Papers")

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=3,
    grouped_task="Explain how unsupervised learning works. Use only the provided content.",
    grouped_properties=["chunk"]
)

for item in response.objects:
    print(item.properties["chunk"])

print(response.generative.text)

2020. Unsupervised cross-lingual representation learning at scale. In Proceedings of the 58th Annual Meeting of the Asso- ciation for Computational Linguistics , pages 8440– 8451, Online. Association for Computational Lin- guistics. Nick Craswell, Bhaskar Mitra, Emine Yilmaz, Daniel Campos, and Ellen M. V oorhees. 2020a. Overview of the trec 2019 deep learning track. Nick Craswell, Bhaskar Mitra, Emine Yilmaz, Daniel Fernando Campos, and Ellen M. V oorhees. 2020b. Overview of the trec 2020 deep learning track. ArXiv , abs/2003.07820. Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. 2019. BERT: Pre-training of deep bidirectional transformers for language under- standing. In Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers)
standard supervised sequence-to-sequence learning or more effectively with reinforcement learning (Ouyang et al., 2022). Con

In [13]:
def paper_rag(paper_id, query, prompt):
    papers = client.collections.get("Papers")
    ten = papers.with_tenant(paper_id)

    response = ten.generate.near_text(
        query=query,
        limit=3,
        grouped_task=prompt + " Use only the provided content.",
        grouped_properties=["chunk"],
    )

    return {
        "title": response.objects[0].properties["title"],
        "source": [p.properties["chunk"] for p in response.objects],
        "generated": response.generative.text
    }

In [14]:
paper_rag(
    "2212-10496",
    "Unsupervised learning",
    "Explain how unsupervised learning works"
)

{'title': 'Precise Zero-Shot Dense Retrieval without Relevance Labels',
 'source': ['2020. Unsupervised cross-lingual representation learning at scale. In Proceedings of the 58th Annual Meeting of the Asso- ciation for Computational Linguistics , pages 8440– 8451, Online. Association for Computational Lin- guistics. Nick Craswell, Bhaskar Mitra, Emine Yilmaz, Daniel Campos, and Ellen M. V oorhees. 2020a. Overview of the trec 2019 deep learning track. Nick Craswell, Bhaskar Mitra, Emine Yilmaz, Daniel Fernando Campos, and Ellen M. V oorhees. 2020b. Overview of the trec 2020 deep learning track. ArXiv , abs/2003.07820. Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. 2019. BERT: Pre-training of deep bidirectional transformers for language under- standing. In Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers)',
  'standard supervised sequence-to-se

In [15]:
papers = client.collections.get("Papers")
papers.tenants.get()

{'2401-00107': TenantOutput(name='2401-00107', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 '2212-10496': TenantOutput(name='2212-10496', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}

## Close the client

In [16]:
client.close()